<a href="https://colab.research.google.com/github/Ashail33/Paper/blob/master/One%20script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1) Environment prep**

In [ ]:
## Set up for data generation
!pip install git+https://github.com/Ashail33/mdcgenpy.git

from  mdcgenpy import clusters as cl
import numpy as np
import pandas as pd
import json
import os
import glob
import re
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
# import cl
from google.colab import drive

# Set up for model creation and evaluation
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("my-app-name").master("local[*]").getOrCreate()



from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from sklearn.base import BaseEstimator, ClusterMixin
import numpy as np

from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.clustering import KMeans as PKMeans
from sklearn.cluster import KMeans as SKKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import numpy as np

from collections import defaultdict
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.metrics import davies_bouldin_score, silhouette_score, calinski_harabasz_score, adjusted_rand_score, normalized_mutual_info_score
from scipy.spatial.distance import cdist
import time
import psutil
import os
import tracemalloc
from google.colab import drive
import glob
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.mixture import GaussianMixture
import json

from sklearn.neighbors import KDTree
import numpy as np
from sklearn.metrics import pairwise_distances

# **2) Create Clustering Dataset**




In [ ]:
# The properties I will be adding to the dataset are:
#     1) Outliers - Binary ( two options) - outliers
#     2) Noise - Binary ( two options) add_noise=0,n_noise=None,
#     3) Number of clusters - I will select three values k
#     4) Number of data points - I will select three values ( 100 000, 1 000 000, 100 000 000)n_samples
#     5) Number of features - I will select five values ( 2, 10, 50, 100, 500) n_feats
#     6) Density - I will select three values compactness_factor
#     --7) Cluster shape - I will select three types - clusters within a radius ,  hollow shaped clusters , s-shaped / c-shaped clusters
#      distributions
#         'uniform': lambda shape, param: np.random.uniform(-param, param, shape),
#     'gaussian': lambda shape, param: np.random.normal(0, param, shape),
#     'logistic': lambda shape, param: np.random.logistic(0, param, shape),
#     'triangular': lambda shape, param: np.random.triangular(-param, 0, param, shape),
#     'gamma': lambda shape, param: np.random.gamma(2 + 8 * np.random.rand(), param / 5, shape),
#     'gap': lambda shape, param: gap(shape, param)

# --8) Missing values - this will need to be created by randomly removing values up to a certain number of columns and records

#Mount drive where you would like to add the data to

drive.mount('/content/gdrive', force_remount=False)

# Set the path to the 'Masters_data' folder in your Google Drive
base_path = '/content/gdrive/MyDrive/'
folder_name = 'Masters_data'
folder_path = os.path.join(base_path, folder_name)

# Create the 'Masters_data' folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Parameters for generating datasets
outliers = [0,500]
noise_options = [(0, None), (10, 100)]
n_clusters = [3, 5, 10]
n_samples = [10000,50000, 100000]
n_feats = [2, 10,100]
compactness_factors = [0.01, 0.5, 1]

# Distributions for cluster shapes
dist_options = ['gaussian']

# Find the last generated dataset
existing_files = glob.glob(os.path.join(folder_path, 'dataset_*.csv'))
dataset_numbers = [int(re.search(r'dataset_(\d+).csv', file).group(1)) for file in existing_files]
if dataset_numbers:
    last_dataset_id = max(dataset_numbers)
else:
    last_dataset_id = -1

metadata = []

# Continue from the last generated dataset
for idx, outlier in enumerate(outliers):
    for idy, (add_noise, n_noise) in enumerate(noise_options):
        for idz, k in enumerate(n_clusters):
            for idw, n_sample in enumerate(n_samples):
                for idv, n_feat in enumerate(n_feats):
                    for idu, compactness_factor in enumerate(compactness_factors):
                        for idt, distribution in enumerate(dist_options):
                            current_dataset_id = (idx * len(noise_options) * len(n_clusters) * len(n_samples) * len(n_feats) * len(compactness_factors) * len(dist_options) +
                                                  idy * len(n_clusters) * len(n_samples) * len(n_feats) * len(compactness_factors) * len(dist_options) +
                                                  idz * len(n_samples) * len(n_feats) * len(compactness_factors) * len(dist_options) +
                                                  idw * len(n_feats) * len(compactness_factors) * len(dist_options) +
                                                  idv * len(compactness_factors) * len(dist_options) +
                                                  idu * len(dist_options) +
                                                  idt)

                            if current_dataset_id <= last_dataset_id:
                                continue

                            file_name = f'dataset_{current_dataset_id}.csv'
                            file_path = os.path.join(folder_path, file_name)
                            distributions = [distribution] * k


                            cluster_gen = cl.ClusterGenerator(
                                          n_samples=n_sample,
                                          outliers=outlier,
                                          n_feats=n_feat,
                                          k=k,
                                          distributions=distributions,
                                          compactness_factor=compactness_factor,

                                          )

                            data = cluster_gen.generate_data(output_file=file_path)

                            dataset_properties = {
                                'id': current_dataset_id,
                                'outliers': outlier,
                                'add_noise': add_noise,
                                'n_noise': n_noise,'n_clusters': k,
                                'n_samples': n_sample,
                                'n_feats': n_feat,
                                'compactness_factor': compactness_factor,
                                'distribution': distribution,
                                'file_path': file_path
                                }

                            metadata.append(dataset_properties)
                            metadata_file_path = os.path.join(folder_path, 'metadata.json')
                            with open(metadata_file_path, 'w') as f:
                                json.dump(metadata, f)


# **3) Create the model functions**

## 3.1) Standard SK-learn models



In [ ]:
from sklearn.cluster import MiniBatchKMeans


## 3.2) Map reduce k-means

In [ ]:
from pyspark.ml.clustering import KMeans as PKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from sklearn.cluster import KMeans as SKKMeans
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
import numpy as np
from sklearn.cluster import KMeans

import numpy as np
from sklearn.datasets import load_iris, load_digits, make_blobs
from sklearn.cluster import KMeans as SKKMeans
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
import matplotlib.pyplot as plt
import timeit
from pyspark.ml.evaluation import ClusteringEvaluator
# Set up SparkSession
spark = SparkSession.builder.appName("MapReduceKMeans").getOrCreate()
sc = spark.sparkContext

from pyspark.ml.clustering import KMeans as PKMeans

class MapReduceKMeans:
    def __init__(self, n_clusters, batch_size, max_iterations):
        self.n_clusters = n_clusters
        self.batch_size = batch_size
        self.max_iterations = max_iterations

    def fit_predict(self, data):
        """
        MapReduce KMeans implementation using PySpark.
        """
        spark = SparkSession.builder.getOrCreate()
        sc = spark.sparkContext
        sc.setLogLevel("ERROR")
        # Convert numpy array to PySpark DataFrame
        df = sc.parallelize(data.tolist()).map(lambda x: (Vectors.dense(x), )).toDF(["features"])
        # Initialize centroids using KMeans++ initialization
        initial_centroids = PKMeans().fit(df.select("features")).clusterCenters()
        global_centroids = initial_centroids
        for i in range(0, data.shape[0], self.batch_size):
            # Split data into batches
            local_data = data[i:i+self.batch_size]
            local_df = sc.parallelize(local_data.tolist()).map(lambda x: (Vectors.dense(x), )).toDF(["features"])
            # Train local KMeans model
            local_kmeans = PKMeans(k=self.n_clusters, initMode="k-means||", maxIter=self.max_iterations, seed=42)
            local_model = local_kmeans.fit(local_df.select("features"))
            local_centroids = local_model.clusterCenters()
            # Update global centroids
            global_centroids = np.vstack([global_centroids, local_centroids])
            global_centroids = SKKMeans(n_clusters=self.n_clusters, random_state=42).fit(global_centroids).cluster_centers_
        # Final KMeans model with global centroids
        kmeans = PKMeans(k=self.n_clusters, initMode="k-means||", maxIter=self.max_iterations, seed=42)
        final_model = kmeans.fit(df.select("features"))
        evaluator = ClusteringEvaluator()
        global_cost = evaluator.evaluate(final_model.transform(df))
        global_centroids = final_model.clusterCenters()
        return KMeans(n_clusters=self.n_clusters, init=global_centroids, n_init=1, random_state=42).fit_predict(data)

Consensus Clustering

In [ ]:
#use this function for weighted consensus method, it has it's issues like the fit to complex shapes irrespective of the underlying model, the spectral clustering of the affinity matrix makes this tricky. This is definitely a shortfall of the method


from sklearn.datasets import make_moons
from sklearn.cluster import SpectralClustering, KMeans
import numpy as np

class ConsensusClustering:
    def __init__(self, n_clusters, model_funcs, weights=None, sampling_rate=0.8, random_state=None):
        self.n_clusters = n_clusters
        self.model_funcs = model_funcs
        self.weights = weights
        self.sampling_rate = sampling_rate
        self.random_state = random_state
        self.co_association_matrix = None

    def fit_predict(self, data):
        n_samples = data.shape[0]
        n_models = len(self.model_funcs)

        if self.weights is None:
            self.weights = np.ones(n_models)

        if len(self.weights) != n_models:
            raise ValueError("The number of weights must be equal to the number of models.")

        self.weights = self.weights / np.sum(self.weights)

        self.co_association_matrix = np.zeros((n_samples, n_samples))

        for i in range(n_models):
            model_func = self.model_funcs[i]
            model = model_func(self.n_clusters)
            sampled_indices = np.random.choice(n_samples, int(n_samples * self.sampling_rate), replace=True)
            sampled_data = data[sampled_indices]

            if callable(model):
                labels = model(sampled_data)
            else:
                labels = model.fit_predict(sampled_data)

            for label in range(self.n_clusters):
                cluster_indices = np.where(labels == label)[0]
                original_indices = sampled_indices[cluster_indices]
                pairs = itertools.combinations(original_indices, 2)
                for x, y in pairs:
                    self.co_association_matrix[x, y] += self.weights[i]
                    self.co_association_matrix[y, x] += self.weights[i]

        spectral_clustering = SpectralClustering(n_clusters=self.n_clusters, affinity='precomputed')
        final_labels = spectral_clustering.fit_predict(self.co_association_matrix)

        return final_labels


# Model functions for k-means and Agglomerative clustering
def kmeans_model_func(n_clusters, random_state):
    return KMeans(n_clusters=n_clusters, random_state=random_state)

def agglomerative_model_func(n_clusters, random_state):
    return AgglomerativeClustering(n_clusters=n_clusters)

def dbscan_model_func(n_clusters):
    return DBSCAN(eps=0.15, min_samples=10)

def optics_model_func(n_clusters, random_state):
    return OPTICS(min_samples=1, xi=0.5, cluster_method='xi', n_jobs=-1)

def optics_DB_Scan_model_func(n_clusters, random_state):
    def optics_instance(data):
        optics = OPTICS(min_samples=1, n_jobs=-1)
        optics.fit(data)
        eps = np.partition(optics.reachability_[optics.ordering_], n_clusters - 1)[n_clusters - 1]
        labels = cluster_optics_dbscan(reachability=optics.reachability_,
                                       core_distances=optics.core_distances_,
                                       ordering=optics.ordering_,
                                       eps=eps)
        return labels

    return optics_instance

# # Define your models in a list
# model_funcs = [KMeans]*5

# # Define the weights for the models
# weights = [0.5, 0.2,0.003,0.4,3]

# # Initialize ConsensusClustering
# consensus_clustering = ConsensusClustering(n_clusters=2, model_funcs=model_funcs, weights=weights, sampling_rate=0.8)

# # Load your data
# data, _ = make_moons(n_samples=200, noise=0.05, random_state=0)

# # Apply consensus clustering
# labels = consensus_clustering.fit_predict(data)

# # `labels` now contains the cluster assignments for each sample in `data`.
# import matplotlib.pyplot as plt

# # Plot the data points with colors corresponding to the assigned cluster
# plt.scatter(data[:, 0], data[:, 1], c=labels, cmap='viridis', alpha=0.6)

# # Title and axis labels
# plt.title('Consensus Clustering Results')
# plt.xlabel('Feature 1')
# plt.ylabel('Feature 2')

# # Show the plot
# plt.show()


S5C

In [ ]:
import numpy as np
import cvxpy as cp
from scipy.sparse import csr_matrix, diags, eye, issparse
from scipy.sparse.linalg import eigsh
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans

class S5C:
    def __init__(self, n_clusters, sparsity_param=0.1, max_iter=1000, tol=1e-4, batch_size=10, random_state=None):
        self.n_clusters = n_clusters
        self.sparsity_param = sparsity_param
        self.max_iter = max_iter
        self.tol = tol
        self.batch_size = batch_size
        self.random_state = random_state
        self.labels_ = None

    def _compute_sparse_representation(self, X, target_idx, selected_samples):
        """Solve Lasso for each data point using CVXPY to get sparse representation."""
        # Exclude the target index from selected_samples
        selected_samples = [idx for idx in selected_samples if idx != target_idx]
        if not selected_samples:
            return np.zeros(X.shape[0])  # Number of samples

        target = X[target_idx, :]  # Shape: (n_features,)

        # Selected samples subset
        X_subset = X[selected_samples, :]  # Shape: (n_samples_subset, n_features)

        # Define the variable
        c = cp.Variable(len(selected_samples))

        # Define the problem
        objective = cp.Minimize(
            0.5 * cp.sum_squares(X_subset.T @ c - target) + self.sparsity_param * cp.norm1(c)
        )
        problem = cp.Problem(objective)

        # Solve the problem using a conic solver
        # problem.solve(solver='ECOS')
        problem.solve(solver=cp.CLARABEL)

        # Check if the problem was solved successfully
        if problem.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
            # If the problem is not solved, return zeros
            return np.zeros(X.shape[0])

        # Get the coefficients
        coef = c.value

        # Create full coefficient vector
        full_coef = np.zeros(X.shape[0])
        full_coef[selected_samples] = coef

        return full_coef

    def _selective_sampling(self, X):
        """Selective sampling with incremental addition based on subgradient violations."""
        n_samples = X.shape[0]
        data, rows, cols = [], [], []

        # Initialize random generator
        rng = np.random.default_rng(self.random_state)

        selected_samples = rng.choice(n_samples, self.batch_size, replace=False).tolist()

        for i in range(n_samples):
            # Compute sparse representation for each data point
            coef = self._compute_sparse_representation(X, i, selected_samples)

            # Append non-zero coefficients to sparse matrix data
            nz_indices = np.nonzero(coef)[0]
            coef_nz = coef[nz_indices]

            # Use absolute values of coefficients to ensure non-negative affinity matrix
            data.extend(np.abs(coef_nz))
            rows.extend(nz_indices)
            cols.extend([i] * len(nz_indices))

            # Calculate subgradient violations
            violations = np.abs(coef_nz) - self.sparsity_param
            max_violation_idx = np.argmax(violations)
            if violations[max_violation_idx] > self.tol:
                violating_sample = nz_indices[max_violation_idx]
                if violating_sample not in selected_samples:
                    selected_samples.append(violating_sample)

        # Create a sparse matrix from sampled data
        C = csr_matrix((data, (rows, cols)), shape=(n_samples, n_samples))
        return C

    def _spectral_clustering(self, W):
        """Spectral clustering with sparse affinity matrix and normalized Laplacian."""
        # Compute the degree matrix
        degrees = np.array(W.sum(axis=1)).flatten()
        # Handle zero degrees to avoid division by zero
        degrees[degrees <= 1e-12] = 1e-12
        D_inv_sqrt = diags(1.0 / np.sqrt(degrees))

        # Compute the normalized Laplacian
        L = eye(W.shape[0]) - D_inv_sqrt @ W @ D_inv_sqrt

        # Ensure L is symmetric
        L = (L + L.T) / 2

        # Compute the smallest k eigenvectors of the Laplacian
        eig_vals, eig_vecs = eigsh(L, k=self.n_clusters, which='SM')
        return eig_vecs

    def fit(self, X):
        """Fit the S5C model."""
        # Ensure X is a NumPy array
        if issparse(X):
            X = X.toarray()

        # Normalize data
        X = normalize(X, norm='l2')

        C = self._selective_sampling(X)
        W = C + C.T  # Symmetrize the affinity matrix

        # Ensure W is non-negative
        W.data = np.maximum(W.data, 0)

        # Perform spectral clustering
        U = self._spectral_clustering(W)
        U_normalized = normalize(U, norm='l2')

        # K-means on spectral embeddings
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=self.random_state)
        self.labels_ = kmeans.fit_predict(U_normalized)

        return self

# **4) Create the Evaluation function**

In [ ]:
from sklearn import metrics

#evaluation code
class Timer:
    def __enter__(self):
        self.start_time = time.monotonic()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.interval = time.monotonic() - self.start_time


class MemoryMonitor:
    def __enter__(self):
        self.process = psutil.Process(os.getpid())
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.memory_info = self.process.memory_info()


class DiskMonitor:
    def __enter__(self):
        self.disk_read_start = psutil.disk_io_counters().read_bytes
        self.disk_write_start = psutil.disk_io_counters().write_bytes
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.disk_read_end = psutil.disk_io_counters().read_bytes
        self.disk_write_end = psutil.disk_io_counters().write_bytes
        self.disk_read_bytes = self.disk_read_end - self.disk_read_start
        self.disk_write_bytes = self.disk_write_end - self.disk_write_start


def compactness(X, labels):
    labels = labels.flatten()
    unique_labels = np.unique(labels)
    centroids = np.array([np.mean(X[labels == i], axis=0) for i in unique_labels])
    distances = cdist(X, centroids)
    return np.sum(np.min(distances, axis=1))


def separation(X, labels):
    labels = labels.flatten()
    unique_labels = np.unique(labels)
    centroids = np.array([np.mean(X[labels == i], axis=0) for i in unique_labels])
    return np.sum(cdist(centroids, centroids))



def batched_silhouette_score(X, labels, batch_size=100000):
    labels = labels.flatten()
    n_samples = len(X)
    n_batches = int(np.ceil(n_samples / batch_size))

    silhouette_sum = 0

    for i in range(n_batches):
        start = i * batch_size
        end = min(start + batch_size, n_samples)

        pairwise_distances = metrics.pairwise_distances(X[start:end], X)

        a = np.array([np.mean(pairwise_distances[i, labels == labels[start + i]]) for i in range(end-start)])
        b = np.array([np.min([np.mean(pairwise_distances[i, labels == cur_label])
                              for cur_label in set(labels) - {labels[start + i]}])
                      for i in range(end-start)])

        silhouette_sum += np.sum((b - a) / np.maximum(a, b))

    return silhouette_sum / n_samples

def dunn_index(X, labels):
    labels = labels.flatten()
    unique_labels = np.unique(labels)
    clusters = [X[labels == i] for i in unique_labels]
    max_intra_cluster_distance = np.max([np.max(cdist(c1, c1)) for c1 in clusters])

    min_inter_cluster_distance = np.inf
    for i, c1 in enumerate(clusters):
        for j, c2 in enumerate(clusters):
            if i < j:
                min_distance = np.min(cdist(c1, c2))
                if min_distance < min_inter_cluster_distance:
                    min_inter_cluster_distance = min_distance

    return min_inter_cluster_distance / max_intra_cluster_distance

from scipy.spatial.distance import pdist, cdist

def batched_dunn_index(X, labels, batch_size=100000):
    n_samples = len(X)
    n_batches = int(np.ceil(n_samples / batch_size))

    max_intra_cluster_distance = -np.inf
    min_inter_cluster_distance = np.inf

    for i in range(n_batches):
        start = i * batch_size
        end = min(start + batch_size, n_samples)

        batch = X[start:end]
        batch_labels = labels[start:end]

        unique_labels = np.unique(batch_labels)

        for label in unique_labels:
            cluster = batch[batch_labels == label]
            intra_cluster_distance = np.max(pdist(cluster))

            if intra_cluster_distance > max_intra_cluster_distance:
                max_intra_cluster_distance = intra_cluster_distance

        for i, c1 in enumerate(unique_labels):
            for j, c2 in enumerate(unique_labels):
                if i < j:
                    inter_cluster_distance = np.min(cdist(X[labels == c1], X[labels == c2]))

                    if inter_cluster_distance < min_inter_cluster_distance:
                        min_inter_cluster_distance = inter_cluster_distance

    return min_inter_cluster_distance / max_intra_cluster_distance


# code to evaluate the clustering algorithms
def evaluate_clustering(algorithm, dataset, true_labels=None, n_runs=5):
    results = defaultdict(list)

    metric_function_map = {
        'cp': compactness,
        'sp': separation,
        'db': davies_bouldin_score,
        'silhouette': silhouette_score,
        'calinski_harabasz': calinski_harabasz_score,
        'ari': adjusted_rand_score,
        'nmi': normalized_mutual_info_score,
        'dvi': dunn_index,
    }

    previous_runs_labels = []
    previous_runs_results = []

    # Start memory monitoring before the loop
    tracemalloc.start()

    for _ in range(n_runs):
        with Timer() as t, DiskMonitor() as d:
            labels = algorithm.fit_predict(dataset)

        # Memory usage tracking
        current_memory, peak_memory = tracemalloc.get_traced_memory()
        results['memory_usage'].append(current_memory)
        results['memory_peak'].append(peak_memory)

        results['runtime'].append(t.interval)
        results['disk_read'].append(d.disk_read_bytes)
        results['disk_write'].append(d.disk_write_bytes)

        if true_labels is not None:
            previous_runs_labels.append(labels)

            metric_results = {}

            # Calculate clustering performance metrics
            for metric_name, metric_function in metric_function_map.items():
                try:
                    value = metric_function(dataset, labels) if metric_name in {'calinski_harabasz', 'davies_bouldin_score', 'silhouette_score', 'cp', 'sp', 'dvi'} else metric_function(true_labels.flatten(), labels.flatten()) if metric_name in {'ari', 'nmi'} else metric_function(dataset, labels.reshape(-1, 1))
                    metric_results[metric_name] = value
                except ValueError as e:
                    print(f"Error calculating {metric_name}: {e}")
                    metric_results[metric_name] = np.nan

            # Add the current run's results to the list of previous runs
            previous_runs_results.append(metric_results)

    # Stop memory monitoring after the loop
    tracemalloc.stop()

    # Aggregate the results over all runs
    for metric_name in metric_function_map.keys():
        metric_values = [run_result[metric_name] for run_result in previous_runs_results]
        results[f"{metric_name}_mean"].append(np.mean(metric_values))
        results[f"{metric_name}_std"].append(np.std(metric_values))
        results[f"{metric_name}_max"].append(np.max(metric_values))
        results[f"{metric_name}_min"].append(np.min(metric_values))

    # Calculate min, max, mean, and std of memory usage over all runs
    memory_usages = np.array(results['memory_usage'])
    memory_stats = {
        'memory_min': np.min(memory_usages),
        'memory_max': np.max(memory_usages),
        'memory_mean': np.mean(memory_usages),
        'memory_std': np.std(memory_usages)
    }

    # Save memory stats
    for stat_name, stat_value in memory_stats.items():
        results[stat_name].append(stat_value)
    results['runtime_mean'] = np.mean(results['runtime'])
    results['runtime_std'] = np.std(results['runtime'])
    results['runtime_max'] = np.max(results['runtime'])
    results['runtime_min'] = np.min(results['runtime'])

    return dict(results)

# **5) Run all models against all data to evaluate metrics**

In [ ]:

import pandas as pd
import json
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
from sklearn import metrics
import gc

from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive', force_remount=False)

# Set the path to the 'Masters_data' folder in your Google Drive
base_path = '/content/gdrive/MyDrive/'
folder_name = 'Masters_data'
folder_path = os.path.join(base_path, folder_name)
n_clusters = 3
batch_size=10
sampling_rate=0.9
\


# Find all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
print(csv_files)
class NumPyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumPyEncoder, self).default(obj)

# Load the metadata
metadata_file_path = os.path.join(folder_path, 'metadata.json')
with open(metadata_file_path, 'r') as f:
    metadata_list = json.load(f)

# Convert the list to a dictionary for easier access
metadata = {item['id']: item for item in metadata_list}



# Loop through all CSV files
for csv_file in csv_files:

    # Use the CSV name as the ID of the dataset
    dataset_id = os.path.splitext(os.path.basename(csv_file))[0]

    # Extract the number from the dataset_id
    dataset_number = int(dataset_id.split('_')[-1])

    # Extract the n_clusters value for the current dataset from metadata
    n_clusters = metadata[dataset_number]['n_clusters']

    # Define the models
    models = {
    'KMeans': {
        'id': 1,
        'function': KMeans(n_clusters=n_clusters)
    },
    'S5C': {
        'id': 2,
        'function': S5C(n_clusters=n_clusters, n_nonzero_coefs=1000, T=1000, B=500)
    },
    'Minibatch_clustering': {
        'id': 3,
        'function': MiniBatchKMeans(n_clusters=n_clusters, batch_size=batch_size, random_state=42)
    },
    'Parallel K-means': {
        'id': 4,
        'function': MapReduceKMeans(n_clusters=n_clusters, batch_size=10, max_iterations=100)
    },
    'Weighted Consensus Clustering': {
        'id': 5,
        'function': ConsensusClustering(n_clusters, model_funcs, weights, sampling_rate)
    }
    }
    # Load the dataset without headers
    df = pd.read_csv(csv_file, header=None)

    # Extract features and true labels
    X = df.iloc[:, :-1].values  # all columns except the last
    y_true = df.iloc[:, -1].values  # last column

    # Loop through each model
    for model_name, model_info in models.items():
        print(model_name)
        print(csv_file)
        model = model_info['function']
        # model.set_params(n_clusters=n_clusters)

        # Evaluate the model on the dataset
        results = evaluate_clustering(algorithm=model, dataset=X, true_labels=y_true, n_runs=1)

        # Add dataset and model information to the results
        results['dataset_id'] = dataset_id
        results['model_id'] = model_info['id']
        results['model'] = model_name
        # Write the results to a new file
        results_file_path = os.path.join(folder_path, f'results_eval.json')
        with open(results_file_path, 'a') as f:
            # And then when dumping the JSON:
            json.dump(results, f, cls=NumPyEncoder)

        results_file_path = os.path.join(folder_path, 'final_results_eval.json')

        # Load the existing results, if any
        if os.path.exists(results_file_path):
            with open(results_file_path, 'r') as f:
                existing_results = json.load(f)
        else:
            existing_results = []

        # Append the new result
        existing_results.append(results)

        # Write the updated results back to the file
        with open(results_file_path, 'w') as f:
            json.dump(existing_results, f, cls=NumPyEncoder)

        # Append the results to the final_evaluation_results file
        # final_results_file_path = os.path.join(folder_path, 'final_evaluation_results.json')
        # if os.path.exists(final_results_file_path):
        #     with open(final_results_file_path, 'r') as f:
        #         final_results = json.load(f)
        # else:
        #     final_results = []

        # final_results.append(results)

        # with open(final_results_file_path, 'w') as f:
        #     json.dump(final_results, f,cls=NumPyEncoder)
    gc.collect()

# **Statistical analysis**

## 6.1) Friedman test

## 6.2) Nemenyi Test